____
## Clean data and write to a file

In [1]:
# This reads in the concatenated data file inoutput_stage1 with fixed formatting 
# and writes to an output_stage2 file

In [2]:
import pandas as pd
import seaborn
import requests
import matplotlib.pyplot as plt

%pylab inline 
%config InlineBackend.figure_format = 'svg'
seaborn.set_style("white")

Populating the interactive namespace from numpy and matplotlib


### 1. Group data by turnstile.  Calculate entries and exits.

In [3]:
df = pd.read_csv('output_stage1.csv', parse_dates=['DATETIME'])
df.drop('Unnamed: 0',axis =1, inplace = True)
df.head()

/Users/user/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME
0,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,00:00:00,REGULAR,13470157,15068968,2018-04-28 00:00:00
1,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,04:00:00,REGULAR,13470247,15069262,2018-04-28 04:00:00
2,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,08:00:00,REGULAR,13470336,15069583,2018-04-28 08:00:00
3,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,12:00:00,REGULAR,13470882,15070622,2018-04-28 12:00:00
4,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,16:00:00,REGULAR,13471623,15071910,2018-04-28 16:00:00


In [4]:
# groupby individual turnstiles
turnstile =['STATION', 'UNIT', 'C/A', 'SCP']

df['ENTRY_DIFF'] = (df.groupby(turnstile)['ENTRIES']
                           .diff()) 
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,ENTRY_DIFF
0,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,00:00:00,REGULAR,13470157,15068968,2018-04-28 00:00:00,NaN
1,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,04:00:00,REGULAR,13470247,15069262,2018-04-28 04:00:00,90.0
2,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,08:00:00,REGULAR,13470336,15069583,2018-04-28 08:00:00,89.0
3,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,12:00:00,REGULAR,13470882,15070622,2018-04-28 12:00:00,546.0
4,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,16:00:00,REGULAR,13471623,15071910,2018-04-28 16:00:00,741.0


In [5]:
df['EXIT_DIFF'] = (df.groupby(turnstile)['EXITS']
                           .diff()) 
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,ENTRY_DIFF,EXIT_DIFF
0,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,00:00:00,REGULAR,13470157,15068968,2018-04-28 00:00:00,NaN,NaN
1,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,04:00:00,REGULAR,13470247,15069262,2018-04-28 04:00:00,90.0,294.0
2,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,08:00:00,REGULAR,13470336,15069583,2018-04-28 08:00:00,89.0,321.0
3,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,12:00:00,REGULAR,13470882,15070622,2018-04-28 12:00:00,546.0,1039.0
4,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,16:00:00,REGULAR,13471623,15071910,2018-04-28 16:00:00,741.0,1288.0


In [6]:
bignum = 100_000

df = df[
    (df["EXIT_DIFF"] > 0) &
    (df["ENTRY_DIFF"] > 0) &
    (df["EXIT_DIFF"] < bignum) &
    (df["ENTRY_DIFF"] < bignum)
]

In [7]:
df.head()

,C/A,UNIT,SCP,STATION,LINENAME,DIVISION,DATE,TIME,DESC,ENTRIES,EXITS,DATETIME,ENTRY_DIFF,EXIT_DIFF
1,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,04:00:00,REGULAR,13470247,15069262,2018-04-28 04:00:00,90.0,294.0
2,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,08:00:00,REGULAR,13470336,15069583,2018-04-28 08:00:00,89.0,321.0
3,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,12:00:00,REGULAR,13470882,15070622,2018-04-28 12:00:00,546.0,1039.0
4,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,16:00:00,REGULAR,13471623,15071910,2018-04-28 16:00:00,741.0,1288.0
5,H007,R248,00-00-00,1 AV,L,BMT,04/28/2018,20:00:00,REGULAR,13472316,15073470,2018-04-28 20:00:00,693.0,1560.0


### 2. Write to csv

In [8]:
df.to_csv('output_stage2.csv')

___
### Next -> select_top_10_stations.ipynb